In [2]:
import psycopg2
import psycopg2.extras
import configparser as configparser

### Configuration of ini file

In [1]:
def parse_ini(section: str) -> dict:
    """
    This function parses ini file for configuration details
    :param section: section to read from ini
    :return: Dictionary of config details
    """
    config = dict()
    parser = configparser.ConfigParser()
    parser.read("database.ini")
    if parser.has_section(section):
        config_items = parser.items(section)
        for item in config_items:
            config[item[0]] = item[1]
    return config

In [3]:
db_config = parse_ini("postgresql")
db_config

{'host': 'localhost',
 'database': 'instacart',
 'user': 'postgres',
 'password': '1997',
 'port': '5432'}

### Function to load the csv

In [4]:
def load_entities(db_config: dict, list_of_csvs: list) -> bool:
    """
    This function insert dataframe to an existing entity table
    :param db_config: Database configuration details
    :param list_of_csvs:
    :return: boolean whether process was done or not
    """
    conn = psycopg2.connect(**db_config, options="-c search_path=dbo,public,instacart")
    cursor = conn.cursor()
    for csv_name, table_name in list_of_csvs:
        try:
            with open(csv_name + '.csv', 'r', encoding='utf-8') as csvfile:
                next(csvfile)  # To skip header row.
                print("Copying from CSV:" + csv_name + " to table:" + table_name + " of DB")
                cursor.copy_from(csvfile, table_name, sep='|', null="\\N")
                conn.commit()
                print("The CSV is loaded to DB")
        except (Exception, psycopg2.DatabaseError) as error:
            print("SQL Exception:" + str(error))
            conn.rollback()
            return False
    cursor.close()
    conn.close()
    return True

### (CSV_name,Table_name) to load onto the database

In [6]:
list_of_csvs = [("aisles_df", "aisles"), ("departments_df", "departments"), ("random_us_addresses_df", "users"), ("products_df", "products"), ("orders_df", "orders"), ("order_products_prior_df", "order_products")]
list_of_csvs

[('random_us_addresses_df', 'users'),
 ('orders_df', 'orders'),
 ('order_products_prior_df', 'order_products')]

In [9]:
load_entities(db_config, list_of_csvs)

Copying from CSV:random_us_addresses_df to table:users of DB
The CSV is loaded to DB
Copying from CSV:orders_df to table:orders of DB
The CSV is loaded to DB
Copying from CSV:order_products_prior_df to table:order_products of DB
The CSV is loaded to DB


True